<a href="https://colab.research.google.com/github/T2718/AI_Make_Image/blob/main/CivitAI12-Mixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# --- モード選択 ---
print("=== モード選択 ===")
print("1: Hugging Faceで画像生成")
print("2: CivitAI（モデルDL / safetensors変換）")
print("3: トークン数を数える")
print("4: 出力フォルダをダウンロード(.zip)")
print("5: 出力フォルダを削除")
mode = input("番号を入力してください: ").strip()

# --- 必要なパラメータを先に全て入力 ---
hf_params = {}
civitai_params = {}

model_list_H = ['stablediffusionapi/eleet-model',
              'stablediffusionapi/brav6',
              'Vsukiyaki/ShiratakiMix',
              'stabilityai/stable-diffusion-xl-base-1.0'
]

model_list_C = [["https://civitai.com/api/download/models/28100?type=Model&format=SafeTensor&size=full&fp=fp16","Anime Pastel Dream"],
                ["https://civitai.com/api/download/models/1410435?type=Model&format=SafeTensor&size=pruned&fp=fp16","WAI-NSFW-illustrious-SDXL v:11.0"]
]



if mode == "1":
  hf_params["model_id"] = input("HuggingFaceのモデルIDを入力（例: runwayml/stable-diffusion-v1-5）: ").strip()
  hf_params["submode"] = input("生成モードを選択（1: text2img, 2: img2img）: ").strip()
  hf_params["prompt"] = input("プロンプトを入力: ").strip()
  hf_params["negative_prompt"] = input("ネガティブプロンプトを入力 : ").strip()
  hf_params["num"] = int(input("生成する画像の枚数 : "))
  hf_params["steps"] = int(input("ステップ数（例: 70）: "))
  hf_params["guidance"] = float(input("guidance scale（例: 7.5）: "))
  if hf_params["submode"] == "2":
    hf_params["strength"] = float(input("strength (例: 0.75) : "))
    hf_params["image_path"] = input("初期画像のパスを入力してください（img2img用）: ")

elif mode == "2":
  print("--- CivitAI モデルの状況は？ ---")
  print("1: まだ何もしていない（DLも変換もしていない）")
  print("2: safetensorsファイルのみアップロード済み（変換が必要）")
  print("3: 変換済みでそのまま使える")
  civitai_params["status"] = input("番号を入力してください: ").strip()
  #civitai_params["model_name"] = input("保存先モデル名（例: models）: ").strip()
  if civitai_params["status"] == "1":
    for k in range(len(model_list_C)):
      print(f"{k+1} : {model_list_C[k][1]}")
    civitai_params["url"] = input("CivitAIのモデル/LoRA ダウンロードURL: ").strip()
    civitai_params["sdxl"] = input("モデルはSDXLですか？ (y/n): ").strip().lower() == "y"
    civitai_params["need_yaml"] = input("v1-inference.yamlが必要ですか？ (y/n): ").strip().lower() == "y"
  elif civitai_params["status"] == "2":
    print("モデルは")
    print("「/content/models/downloaded_model.safetensors」")
    print("に保存してください。")
    civitai_params["sdxl"] = input("モデルはSDXLですか？ (y/n): ").strip().lower() == "y"
    civitai_params["need_yaml"] = input("v1-inference.yamlが必要ですか？ (y/n): ").strip().lower() == "y"
  else:
    civitai_params["sdxl"] = input("モデルはSDXLですか？ (y/n): ")
  civitai_params["need_pipe"] = input("pipelineをインストールしますか？ (y/n): ").strip().lower() == "y"
  civitai_params["submode"] = input("生成モードを選択 (1: text2img, 2: img2img) : ").strip()
  civitai_params["prompt"] = input("プロンプトを入力してください: ")
  civitai_params["negative_prompt"] = input("ネガティブプロンプトを入力 : ").strip()
  civitai_params["num"] = int(input("生成する画像の枚数 : "))
  civitai_params["steps"] = int(input("ステップ数（例: 70）: "))
  if civitai_params["submode"] == "2":
    civitai_params["strength"] = float(input("strength (例: 0.75)(img2img用) : "))
    civitai_params["image_path"] = input("初期画像のパスを入力してください（例: /content/your_img.png）: ")
  civitai_params["lora_path"] = input("LoRAを使用する場合はpathを(使わない場合は無入力)")
  if civitai_params["lora_path"] == "":
    civitai_params["lora_path"] = None

elif mode == "3":
  token_prompt = input("プロンプトを入力してください（トークン数確認）: ").strip()

elif mode == "4":
  print("少々お待ちください。")

elif mode == "5":
  confirm_delete = input("本当にoutputフォルダを削除しますか？ (y/n): ").strip().lower()

# --- 必要なinstallはここで実行（全てのinput終了後） ---
print("インストールを開始します...")
!pip install diffusers transformers accelerate safetensors

# --- 各モード実行 ---
if mode == "1":
  from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
  from PIL import Image
  import torch
  import os
  torch_dtype = torch.float16
  if hf_params["submode"] == "1":
    pipe = StableDiffusionPipeline.from_pretrained(
      hf_params["model_id"],
      torch_dtype=torch.float16,
      safety_checker=None  # Safe checkを無効に
    )
    pipe = pipe.to("cuda")  # GPUが必要です
  else:
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
        hf_params["model_id"],
        torch_dtype=torch.float16,
        safety_checker=None
    )
    pipe = pipe.to("cuda")
  if hf_params["submode"] != "1":
    init_image = Image.open(hf_params["image_path"]).convert("RGB").resize((512, 512))
  for k in range(hf_params["num"]):
    if hf_params["submode"] == "1":

      image = pipe(hf_params["prompt"],
                   negative_prompt=hf_params["negative_prompt"],
                   num_inference_steps=hf_params["steps"],
                   guidance_scale=hf_params["guidance"],
                   added_cond_kwargs={}).images[0]
    else:
      #init_image = Image.open(hf_params["image_path"]).convert("RGB").resize((512, 512))
      image = pipe(prompt=hf_params["prompt"],
                   negative_prompt=hf_params["negative_prompt"],
                   num_inference_steps=hf_params["steps"],
                   image=init_image,
                   strength=hf_params["strength"],
                   guidance_scale=hf_params["guidance"]).images[0]

    os.makedirs("output", exist_ok=True)
    filename = f"output/huggingface_result"+str(k+1)+".png"
    image.save(filename)
  print("画像を保存しました")

elif mode == "2":
  import os
  import torch
  import re
  if civitai_params["submode"] == "2":
    from PIL import Image
    from IPython.display import display


  base_dir = "/content/models"
  os.makedirs(base_dir, exist_ok=True)
  download_path = "/content/models/downloaded_model.safetensors"

  if civitai_params["status"] == "1":
    regex = r'([0-9]+)'
    regex2 = r'#(\d+)'
    if bool(re.match(regex, civitai_params['url'])):
      civitai_params['url'] = model_list_C[int(civitai_params['url'])-1][0]
    else:
      match = re.fullmatch(regex2, civitai_params['url'])
      if match:
        civitai_params['url'] = 'https://civitai.com/api/download/models/'+match.group(1)


    model_url = civitai_params["url"]

    print(model_url)

    print(f"Downloading Checkpoint model to {download_path}...")
    !wget -O {download_path} "{model_url}"
    print(f"Download of safetensorFile complete!")

    # Checkpointモデルのパスを記録
    checkpoint_path = download_path
  lora_path = civitai_params["lora_path"] # LoRAはダウンロードしないのでNone


  if civitai_params["status"] == "1" or civitai_params["status"] == "2":
    if civitai_params["need_yaml"]:
      !wget -O /content/models/v1-inference.yaml \
https://raw.githubusercontent.com/CompVis/stable-diffusion/main/configs/stable-diffusion/v1-inference.yaml
      #!wget -q https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-inference.yaml
    if not civitai_params["sdxl"]:
      !mkdir /content/converted
      !wget -O convert_original_stable_diffusion_to_diffusers.py \
      https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_original_stable_diffusion_to_diffusers.py
      os.makedirs(base_dir, exist_ok=True)
      !python convert_original_stable_diffusion_to_diffusers.py \
        --checkpoint_path ./models/downloaded_model.safetensors \
        --original_config_file ./models/v1-inference.yaml \
        --dump_path ./converted/diffusers_model \
        --from_safetensors

  #pipe = None
  if civitai_params["need_pipe"]:
    if civitai_params["sdxl"]:
      if civitai_params["submode"] == "1":
        from diffusers import StableDiffusionXLPipeline
        pipe = StableDiffusionXLPipeline.from_single_file(
            download_path,
            torch_dtype=torch.float16, # fp16モデルなので必須
            use_safetensors=True,      # safetensors形式であることを明示
            safety_checker=None
        ).to("cuda")
      else:
        from diffusers import StableDiffusionXLImg2ImgPipeline
        from PIL import Image
        import torch

        pipe = StableDiffusionXLImg2ImgPipeline.from_single_file(
            download_path,
            torch_dtype=torch.float16,
            use_safetensors=True,
            safety_checker=None,
        ).to("cuda")

      # VRAM節約と高速化のための設定（非常に重要）
      #pipe.enable_xformers_memory_efficient_attention()

      # 最も重要なVRAM節約策：モデルの一部をCPUにオフロード
      # VRAMが16GB以下のGPUでは、SDXLを動かすためにほぼ必須です
      pipe.enable_model_cpu_offload()
    else:
      from diffusers import StableDiffusionPipeline
      #pipe = StableDiffusionPipeline.from_pretrained(base_dir, torch_dtype=torch.float16).to("cuda")
      pipe = StableDiffusionPipeline.from_pretrained(
          "/content/converted/diffusers_model",
          torch_dtype=torch.float16,
          use_safetensors=False,
          safety_checker=None
        ).to("cuda")
    # パイプラインを作成した直後に以下を追加します
    if lora_path:
      pipe.load_lora_weights(lora_path, use_safetensors=True)

  if civitai_params["submode"] == "1":
  #pipe.enable_xformers_memory_efficient_attention()
    for k in range(civitai_params["num"]):
      image = pipe(prompt=civitai_params["prompt"],
          negative_prompt=civitai_params["negative_prompt"],
          num_inference_steps=civitai_params["steps"]
      ).images[0]

      os.makedirs("output/t2i/"+civitai_params["prompt"][0:10], exist_ok=True)
      image.save("output/t2i/"+civitai_params["prompt"][0:10]+"/"+str(k+1)+".png")
    print("画像を保存しました: output/t2i/"+civitai_params["prompt"][0:10])
  else:
    init_image = Image.open(civitai_params["image_path"]).convert("RGB").resize((1024, 1024))
    #display(init_image)
    for k in range(civitai_params["num"]):
      image = pipe(prompt=civitai_params["prompt"],
          negative_prompt=civitai_params["negative_prompt"],
          image=init_image,
          num_inference_steps=civitai_params["steps"],
          strength=civitai_params["strength"],
      ).images[0]

      os.makedirs("output/i2i/"+civitai_params["prompt"][0:10], exist_ok=True)
      image.save("output/i2i/"+civitai_params["prompt"][0:10]+"/"+str(k+1)+".png")
    print("画像を保存しました: output/i2i/"+civitai_params["prompt"][0:10])

elif mode == "3":
  from transformers import CLIPTokenizer
  tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
  tokens = tokenizer(token_prompt)["input_ids"]
  print(f"トークン数: {len(tokens)}")

elif mode == "4":
  import shutil
  from google.colab import files

  # 圧縮したいディレクトリのパス
  folder_path = '/content/output'
  zip_filename = folder_path + '.zip'

  # ディレクトリを zip 形式で圧縮
  shutil.make_archive(folder_path, 'zip', folder_path)

  # Colab にダウンロードリンクを表示
  files.download(zip_filename)


elif mode == "5":
  import shutil
  if confirm_delete == "y" and os.path.exists("output"):
    shutil.rmtree("output")
    print("outputフォルダを削除しました")
  else:
    print("キャンセルされました")


=== モード選択 ===
1: Hugging Faceで画像生成
2: CivitAI（モデルDL / safetensors変換）
3: トークン数を数える
4: 出力フォルダをダウンロード(.zip)
5: 出力フォルダを削除
番号を入力してください: 4
本当にoutputフォルダを削除しますか？ (y/n): y
インストールを開始します...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>